In [27]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from keras.models import Model, Sequential, load_model, Input
from keras.layers import (Conv2D, Dense, MaxPooling2D, LeakyReLU, Reshape,
                          Flatten, Dropout, BatchNormalization)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
import efficientnet.keras as efn
from sklearn.utils import class_weight

In [18]:
IMAGE_SIZE=500
BATCH_SIZE=32
REG=0.0005
EPOCHS=30
TRAIN_IMAGE_PATH = "jpeg/train/"
TEST_IMAGE_PATH = "jpeg/test/"

In [19]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [20]:
train_df = train_df[["image_name", "target"]]
test_df = train_df[["image_name"]]

In [21]:
train_df["image_name"] = train_df["image_name"].apply(lambda x: TRAIN_IMAGE_PATH+x+".jpg")
test_df["image_name"] = test_df["image_name"].apply(lambda x: TEST_IMAGE_PATH+x+".jpg")

c:\users\ashuk\.conda\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
#train_df_copy, val_df = train_test_split(train_df_copy, test_size=0.1, random_state=45)
target_0 = len(train_df[train_df["target"]==0])
target_1 = len(train_df[train_df["target"]==1])
no_0 = np.random.randint(low=0, high=target_0, size=(int(0.9*(target_0)),))
no_1 = np.random.randint(low=0, high=target_1, size=(int(0.9*(target_1))))
x_train = train_df.iloc[np.concatenate([no_0, no_1])]
val_df = train_df.drop(train_df.index[np.concatenate([no_0, no_1])])
train_df = x_train

In [35]:
def decode_image(filename, label=None, image_size=(IMAGE_SIZE, IMAGE_SIZE)):
    bits = tf.io.read_file(filename)
    data = tf.image.decode_jpeg(bits, channels=3)
    data = tf.cast(data, tf.float32)/255.0
    data = tf.image.resize(data, image_size)
    if label is None:
        return data
    else:
        return data, label
    
def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if label is None:
        return image
    else:
        return image, label
    

In [36]:
train_data = (
    tf.data.Dataset
    .from_tensor_slices((train_df["image_name"].values, train_df["target"].values))
    .map(decode_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(data_augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
)

val_data = (
        tf.data.Dataset
    .from_tensor_slices((val_df["image_name"].values, val_df["target"].values))
    .map(decode_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
)

test_data = (
    tf.data.Dataset
    .from_tensor_slices((test_df["image_name"].values))
    .map(decode_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
)

In [25]:
lr = ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.5,
    patience=5,
    min_lr=0.00001,
    verbose=1
)
es = EarlyStopping(
    monitor="val_loss",
    patience=7
)


In [28]:
model_ef7_ns = efn.EfficientNetB7(include_top=False, weights="noisy-student", input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
model_ef7_ns.trainable=False
model = Conv2D(filters=32, kernel_size=(3,3), data_format="channels_last" ,activation="relu", kernel_regularizer=l2(REG))(model_ef7_ns.output)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Conv2D(filters=32, kernel_size=(3,3), activation="relu", kernel_regularizer=l2(REG))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = MaxPooling2D(pool_size=(2,2), padding="SAME")(model)
model = Dropout(0.25)(model)
model = Flatten()(model)
model = Dense(256, activation="relu")(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Dropout(0.5)(model)
model = Dense(64, activation="relu")(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
output = Dense(1, activation="sigmoid")(model)
model_d121 = Model(inputs=model_ef7_ns.input, outputs=output)
model_d121.summary()
model_d121.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 500, 500, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 250, 250, 64) 1728        input_3[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 250, 250, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 250, 250, 64) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

Total params: 65,157,073
Trainable params: 64,845,585
Non-trainable params: 311,488
__________________________________________________________________________________________________


In [29]:
w_0, w_1 = class_weight.compute_class_weight("balanced", np.unique(train_df["target"]), train_df["target"])

class_weight = {
    0: w_0,
    1: w_1
}

In [37]:
history = model_d121.fit(train_data,
                                   epochs=EPOCHS,
                                   steps_per_epoch=len(train_df)//BATCH_SIZE,
                                   callbacks = [lr, es],
                                   validation_data = val_data,
                                   validation_steps=25,
                                   class_weight=class_weight
                                  )

AttributeError: 'BatchDataset' object has no attribute 'ndim'

In [ ]:
test_generator.reset()
pred = model_d121.predict(test_generator, steps = len(test_generator), verbose=1)

pred

sub = pd.read_csv("sample_submission.csv")

sub

sub["target"] = pred

#sub.to_csv("d121_1conv2d.csv", index=False)

sub.to_csv("ef7_ns_conv2d.csv", index=False)

In [34]:
data_augment(decode_image("jpeg/train/ISIC_1936882.jpg"))

<tf.Tensor: shape=(500, 500, 3), dtype=float32, numpy=
array([[[0.18235295, 0.11568628, 0.14705883],
        [0.18333334, 0.10490196, 0.14803922],
        [0.17254902, 0.12156863, 0.15686275],
        ...,
        [0.3156863 , 0.16764706, 0.2       ],
        [0.33137256, 0.15686275, 0.19803922],
        [0.30784315, 0.14705883, 0.17843138]],

       [[0.18137255, 0.11470588, 0.14607844],
        [0.18921569, 0.11078431, 0.15392157],
        [0.17058824, 0.11960784, 0.15490197],
        ...,
        [0.3264706 , 0.15196079, 0.19313726],
        [0.30588236, 0.15784314, 0.19019608],
        [0.3       , 0.15882353, 0.18431373]],

       [[0.17843138, 0.11568628, 0.15490197],
        [0.18431373, 0.1254902 , 0.15294118],
        [0.17058824, 0.11176471, 0.1392157 ],
        ...,
        [0.327451  , 0.15882353, 0.19411765],
        [0.3       , 0.15490197, 0.18235295],
        [0.30392158, 0.14705883, 0.17843138]],

       ...,

       [[0.18137255, 0.10294118, 0.11078431],
        [0.19

In [31]:
train_df

,image_name,target
6142,jpeg/train/ISIC_1936882.jpg,0
29321,jpeg/train/ISIC_8863678.jpg,0
5500,jpeg/train/ISIC_1744817.jpg,0
21150,jpeg/train/ISIC_6421507.jpg,0
9210,jpeg/train/ISIC_2856521.jpg,0
...,...,...
470,jpeg/train/ISIC_0249560.jpg,0
538,jpeg/train/ISIC_0271017.jpg,0
415,jpeg/train/ISIC_0236510.jpg,0
439,jpeg/train/ISIC_0240628.jpg,0
